Upon completing the code and data sets that were posted under this week, continue using the below codes to construct the simplified data‐driven cost matrix as follows:  
a.	Obtain the mean of the Sales per Visit variable from the training data set and set the negative of that value to be the “cost” of a true positive. Let the false positive cost equal $10.  

In [12]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier
from google.colab import files

uploaded = files.upload()

clothing_train = pd.read_csv("clothing_data_driven_training", sep="\t")
clothing_test = pd.read_csv("clothing_data_driven_test", sep="\t")

#The 'Response' column is extracted from the clothing_train and values is stored in the variable y.
y = clothing_train[['Response']]

#New DataFrame named 'X' that contains columns.
X = clothing_train[['Days since Purchase','Number of Purchase Visits','Sales per Visit']]

Saving clothing_data_driven_training to clothing_data_driven_training (4)


In [13]:
# a
# The mean of column 'Sales per Visit' is rounding and calculating.
# the round function is used to round a number in certain number of digits (Amos, 2023).
sales_mean = round(clothing_train['Sales per Visit'].mean(),2)
tp_cost = sales_mean - (sales_mean*2)
fp_cost = 10
# Displaying the outcome which was the Mean Sales.
sales_mean

113.58

b.	Construct the appropriate cost matrix, and simplify it, to obtain the simplified data‐driven cost matrix.

In [22]:
# b
# Creating a Cost Matrix where values in 0 represents the cost of negative class while values in 1 shows the costs of positive class.
data = {'0': [0, 0],
        '1': [fp_cost , tp_cost ]
        }

#DataFrame is a two-dimensional tabular data structure in data manipulation and analysis Python library (Palomino & Wasser, 2020).
cost_matrix = pd.DataFrame (data, columns = ['0','1'])

#The original cost matrix by fp_cost or false positive cost is partitioned by the adjusted cost matrix.
cost_matrix_adj = cost_matrix / fp_cost

#The iloc indexer in pandas is utilized in integer-location based indexing of a DataFrame (Dham, 2023).
cost_matrix_adj.iloc[1,0] = cost_matrix_adj.iloc[1,1] - (cost_matrix_adj.iloc[1,1]*2)
cost_matrix_adj.iloc[1,1] = cost_matrix_adj.iloc[1,0 ]- (cost_matrix_adj.iloc[1,0]*2)

Using the training set, build a C5.0 model (Model 3) to predict a customer's Response using Days since Purchase, # of Purchase Visits, and Sales per Visit, using the data‐driven cost matrix.

In [15]:
# The costs between keys 0 and 1 represented by cost_matrix dictionary.
cost_matrix={0:cost_matrix_adj.iloc[0,1],1:cost_matrix_adj.iloc[1,0]}

# A decision tree classifier 'c50_03' was created with parameters.
c50_03=DecisionTreeClassifier(criterion="entropy", min_samples_split= 75, max_leaf_nodes = 5, class_weight=cost_matrix).fit(X,y)

Populate the Model Evaluation Table with the evaluation measures for Model 3, using the data‐driven cost matrix.

In [16]:
# To predict on clothing_test with the following columns, the c50_03 is used.
#Predict function makes predictions on new or uncovered data (Mulani, 2020).
c50_03_predict=c50_03.predict(clothing_test[['Days since Purchase','Number of Purchase Visits','Sales per Visit']])

# Creating the contingency table to compare actual vs. predicted responses.
# The crosstab is convenient way to use in computing a cross-tabulation of two or more factors in a DataFrame (Gupta, 2023).
con_table_3 = pd.crosstab(clothing_test['Response'], c50_03_predict)
con_table_3

col_0,0,1
Response,,
0,4897,4717
1,266,1674


In [20]:
def model_eval_cost(matrix, model_name, fn_cost, tn_cost, fp_cost, tp_cost):
  # This part is extracting the values from contingency table

    tn = matrix.iloc[0, 0] # tn represents the True Negatives
    tp = matrix.iloc[1, 1] # tp represents the True Positives
    fn = matrix.iloc[1, 0] # fn represents the False Negatives
    fp = matrix.iloc[0, 1] # fp represents the False Positives

    # Getting the sum or totals for different metrics
    #tap represents the total number of instances

    tap = fn + tp # Total number of instances in Actual Positives
    tan = tn + fp # Total number of instances in Total Actual Negatives
    tpn = tn + fn # Total number of instances in Predicted Negatives
    tpp = fp + tp # Total number of instances in Predicted Positives

    # The calculation of precision is likewise the ratio of true positives to the total predicted positives.

    precision = tp / tpp if tpp != 0 else 0
    recall = tp / tap if tap != 0 else 0

    # Getting the total or sum of True Negatives, True Positives, False Negatives, and False Positives
    total = tn + tp + fn + fp

    data = [
        round((tp + tn) / total, 4),
        round(1 - ((tp + tn) / total), 4),
        round(tp / tap, 4),
        round(tn / tan, 4),
        round(precision, 4),
        round(2 * (precision * recall) / (precision + recall), 4) if (precision + recall) != 0 else 'Na',
        round(5 * (precision * recall) / ((4 * precision) + recall), 4) if (precision + recall) != 0 else 'Na',
        round(1.25 * (precision * recall) / ((.25 * precision) + recall), 4) if (precision + recall) != 0 else 'Na',
        round((fn * fn_cost) + (tn * tn_cost) + (fp * fp_cost) + (tp * tp_cost), 2),
        round(((fn * fn_cost) + (tn * tn_cost) + (fp * fp_cost) + (tp * tp_cost)) / total, 2)
    ]

    return pd.DataFrame(data, columns=[model_name],
                        index=['Accuracy', 'Error Rate', 'Sensitivity', 'Specificity', 'Precision', 'F1', 'F2', 'F3',
                               'Overall Cost', 'Profit per Customer'])

Model_3 = model_eval_cost(con_table_3, model_name='Model_3', fn_cost=0, tn_cost=0, fp_cost=10, tp_cost=-40)
print(Model_3)


                        Model_3
Accuracy                 0.5687
Error Rate               0.4313
Sensitivity              0.8629
Specificity              0.5094
Precision                0.2619
F1                       0.4019
F2                       0.5915
F3                       0.3043
Overall Cost        -19790.0000
Profit per Customer     -1.7100


In [21]:
import datetime
import socket
def get_Host_name_IP():
    try:
        host_name = socket.gethostname()
        host_ip = socket.gethostbyname(host_name)
        print("Hostname:",host_name)
        print("IP Address:",host_ip)
    except:
        print("No visible IP Address")
get_Host_name_IP()
now = datetime.datetime.now()
print ("Time Stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))

Hostname: d401aa15b01f
IP Address: 172.28.0.12
Time Stamp: 2024-01-18 13:58:01


**References:**

Amos, D. (2023, July 03). How to Round Numbers in Python. Retrieved from Real Python: https://realpython.com/python-rounding/

Dham, M. (2023, February 25). iloc() Function in Python. Retrieved from PrepBytes: https://www.prepbytes.com/blog/python/iloc-function-in-python/

Gupta, R. (2023, May 06). The Power of Crosstab Function in Pandas for Data Analysis and Visualization. Retrieved from Medium: https://medium.com/geekculture/the-power-of-crosstab-function-in-pandas-for-data-analysis-and-visualization-6c085c269fcd

Mulani, S. (2020, October 13). Python predict() function – All you need to know! Retrieved from AskPython: https://www.askpython.com/python/examples/python-predict-function

Palomino, J., & Wasser, L. (2020, November 12). Lesson 4. Select Data From Pandas Dataframes. Retrieved from Earth Lab: https://www.earthdatascience.org/courses/intro-to-earth-data-science/scientific-data-structures-python/pandas-dataframes/indexing-filtering-data-pandas-dataframes/



